In [19]:
import numpy as np
import pandas as pd
import random

# 数据加载，划分测试集，训练集，预处理

In [20]:
class Get_train_test(object):
    def __init__(self,path):
        self.path=path
        
    def Norm(self,data):
        '''
        标准化
        '''
        
        
        data_max=data.max(axis=0)
        data_min=data.min(axis=0)
        
        final_data=(data-data_min)/(data_max-data_min)
        
        return final_data
    
   
    def get_random_number(self):
        list_ram_num=[]
        
        while len(list_ram_num)<50:
            
            num=random.randint(0,207)
            
            list_ram_num.append(num)
            
            list_ram_num=list(set(list_ram_num))
        return np.array(list_ram_num)
        
    def get_Data(self):
        Data=pd.read_csv(self.path,header=None)
        data=np.array(Data)[:,:-1]
        labels=np.array(Data)[:,-1]
        data=self.Norm(data)
        random_num=self.get_random_number()
        
        test_ran=list(range(50))
        np.random.shuffle(test_ran)
        train_ran=list(range(158))
        np.random.shuffle(train_ran)
        data_test=data[random_num][test_ran]
        label_test=labels[random_num][test_ran]
      
        data_train=np.delete(data,random_num,axis=0)[train_ran]
        label_train=np.delete(labels,random_num,axis=0)[train_ran]
        
        return data_train,label_train,data_test,label_test
  

In [21]:
data_train,label_train,data_test,label_test=Get_train_test('./相关资料/sonar.csv').get_Data()

In [22]:
data1=np.concatenate((data_train,label_train.reshape(-1,1)),axis=1)

# 决策树算法，后剪枝

In [23]:
from math import log
from collections import Counter
import re

In [24]:
from math import log
from collections import Counter
import numpy as np

class DecisionTree(object):
    def __init__(self,algor=None,detla=0.01,root='0'):
        self.algor=algor
        self.detla=detla
        self.root=root
    #计算熵值
    def cal_entropy(self,labels):
        entropy=0.0
        dic_1={}#计算每种标签出现的此数
        for item in labels:
            dic_1[item]=dic_1.get(item,0)+1
        for itemN in dic_1:#计算每个标签占的比率
            prod=dic_1[itemN]/len(labels)#概率
            entropy-=prod*log(prod,2)#熵值
        return entropy
    #计算给定特征的各种划分下的最小熵值
    def feaX_entropy(self,dataset,i):
        entropy=0.0
        feaX_labels=[[x[i],x[-1]] for x in dataset]#将待选择的特征与标签选出
        feaX=[x[i] for x in dataset]
        def takefirst(elem):
            return elem[0]
        feaX_labels.sort(key=takefirst)
        feaX.sort()
        candidate=[]#分割值的候选处
        labels=[]
        labels1=[]
        entropy_list=[]
        for i in range(len(feaX)-1):
            candidate.append((feaX[i]+feaX[i+1])/2)
        for candidate_sample in candidate:#计算每个分割值对应下的熵
            count=0
            for feaX_labels_sample in feaX_labels:
                if candidate_sample<=feaX_labels_sample[0] :
                    count+=1
                    labels.append(feaX_labels_sample[-1])
                else:
                    labels1.append(feaX_labels_sample[-1])
            entropy=count/len(feaX)*self.cal_entropy(labels)+(len(feaX)-count)/len(feaX)*self.cal_entropy(labels1)
            entropy_list.append(entropy)#收成列表
            #返回最小熵值，索引，和分割值
        return min(entropy_list),entropy_list.index(min(entropy_list)),candidate[entropy_list.index(min(entropy_list))]
      
class ID3(DecisionTree):
    def __init__(self,algor='ID3',detla=0.05):
        super(ID3,self).__init__()
        self.algor=algor
        self.detla=detla
    def info_gain(self,dataset,i):#计算信息增益
        labels=[x[-1] for x in dataset]
        entropy,index,candidate_final=self.feaX_entropy(dataset,i)
        info_gain=self.cal_entropy(labels)-entropy
        return info_gain
    def find_max_info_gain(self,dataset):#遍历所有特征下的信息增益，返回最大的信息增益
        num_sample=len(dataset[0])-1
        find_max=[]
        for i in range(num_sample):
            find_max.append(self.info_gain(dataset,i))
        return find_max.index(max(find_max)),max(find_max)
    #构建决策树函数 返回列表
    def creatdisiontree(self,dataset,fea_labels,detle):
        labels=[x[-1]for x in dataset]#判断标签是否一致，若一致直接返回标签值
        if len(list(set(labels)))==1:
            return labels[0]
        max_entropy_index,max_info_gain=self.find_max_info_gain(dataset)
        if max_info_gain<detle:#判断最大信息增益是否小于给定值，若小于则不再生成树
            label_count=Counter(labels)
            return max(label_count.items(),key=lambda x:x[1])[0]
        else:
            fea_dic_val={}#用于收集分支的标志和下面的节点
            fea_dic={}#用于收集节点和分支
            feaX_labels=[[x[max_entropy_index],x[-1]]for x in dataset]
            feaX=[x[max_entropy_index] for x in dataset]
            fea_labels_new=[]#产生新的特征列表
            fea_labels_new.extend(fea_labels[:max_entropy_index]+fea_labels[max_entropy_index+1:])
            entropy,index,candidate_final=self.feaX_entropy(dataset,max_entropy_index)
            tem_dict={'min_than %s'% (candidate_final):[],'max_than %s'%(candidate_final):[]}
            for sample in feaX:#利用最终选取的划分值对样本进行划分
                if sample<=candidate_final:
                    tem_dict['min_than %s'% (candidate_final)].append(sample)
                if sample>candidate_final:
                    tem_dict['max_than %s'%(candidate_final)].append(sample)
            data_new1=[]
            data_new2=[]
            for item in dataset:#对样本进行划分，并除去此特征生成新的特征列表
                
                if item[max_entropy_index] in tem_dict['min_than %s'% (candidate_final)]:
                  
                    data_new1.append(list(item[:max_entropy_index])+list(item[max_entropy_index+1:]))
                else:
                    data_new2.append(list(item[:max_entropy_index])+list(item[max_entropy_index+1:]))
            if len(data_new1)==0:
                label1=[y[-1]for y in data_new2]
                label1_count=Counter(label1)
                return max(label1_count.items(),key=lambda y:y[1])[0]
            if len(data_new2)==0:
                label2=[y[-1]for y in data_new1]
                label2_count=Counter(label2)
                return max(label2_count.items(),key=lambda y:y[1])[0]
            if len(data_new1)!=0 and len(data_new2)!=0:#如果不是空集，对样本划分，并迭代
                fea_dic_val['min_than %s'% (candidate_final)]=fea_dic_val.get('min_than %s'% (candidate_final),self.creatdisiontree(data_new1,fea_labels_new,detle))
                fea_dic_val['max_than %s'%(candidate_final)]=fea_dic_val.get('max_than %s'%(candidate_final),self.creatdisiontree(data_new2,fea_labels_new,detle))
                #生成节点和分叉
                fea_dic[fea_labels[max_entropy_index]]=fea_dic.get(fea_labels[max_entropy_index],fea_dic_val)
        return fea_dic

In [25]:
fea_dic_0=ID3().creatdisiontree(dataset=data1,fea_labels=np.array(range(60)).tolist(),detle=0.05)
fea_dic_1=ID3().creatdisiontree(dataset=data1,fea_labels=np.array(range(60)).tolist(),detle=0.07)
fea_dic_2=ID3().creatdisiontree(dataset=data1,fea_labels=np.array(range(60)).tolist(),detle=0.1)
fea_dic_3=ID3().creatdisiontree(dataset=data1,fea_labels=np.array(range(60)).tolist(),detle=0.14)
fea_dic_4=ID3().creatdisiontree(dataset=data1,fea_labels=np.array(range(60)).tolist(),detle=0.16)

# 生成的决策树

In [26]:
fea_dic_0

{10: {'min_than 0.26003119240039696': {21: {'min_than 0.7646968612616297': {2: {'min_than 0.1307490144546649': {43: {'min_than 0.048119041484153566': 'M',
        'max_than 0.048119041484153566': 'R'}},
      'max_than 0.1307490144546649': {3: {'min_than 0.11412268188302425': 'R',
        'max_than 0.11412268188302425': {24: {'min_than 0.3669057377049181': 'R',
          'max_than 0.3669057377049181': 'M'}}}}}},
    'max_than 0.7646968612616297': {41: {'min_than 0.23492063492063497': {8: {'min_than 0.23463645787057602': 'R',
        'max_than 0.23463645787057602': 'M'}},
      'max_than 0.23492063492063497': {30: {'min_than 0.5552588555858311': 'M',
        'max_than 0.5552588555858311': 'R'}}}}}},
  'max_than 0.26003119240039696': {16: {'min_than 0.717231374987048': {34: {'min_than 0.3648869796461082': {17: {'min_than 0.09506493506493506': 'R',
        'max_than 0.09506493506493506': 'M'}},
      'max_than 0.3648869796461082': {17: {'min_than 0.5362597402597402': {11: {'min_than 0.423

In [27]:
fea_dic_1

{10: {'min_than 0.26003119240039696': {21: {'min_than 0.7646968612616297': {2: {'min_than 0.1307490144546649': {43: {'min_than 0.048119041484153566': 'M',
        'max_than 0.048119041484153566': 'R'}},
      'max_than 0.1307490144546649': {3: {'min_than 0.11412268188302425': 'R',
        'max_than 0.11412268188302425': {24: {'min_than 0.3669057377049181': 'R',
          'max_than 0.3669057377049181': 'M'}}}}}},
    'max_than 0.7646968612616297': {41: {'min_than 0.23492063492063497': {8: {'min_than 0.23463645787057602': 'R',
        'max_than 0.23463645787057602': 'M'}},
      'max_than 0.23492063492063497': {30: {'min_than 0.5552588555858311': 'M',
        'max_than 0.5552588555858311': 'R'}}}}}},
  'max_than 0.26003119240039696': {16: {'min_than 0.717231374987048': {34: {'min_than 0.3648869796461082': {17: {'min_than 0.09506493506493506': 'R',
        'max_than 0.09506493506493506': 'M'}},
      'max_than 0.3648869796461082': {17: {'min_than 0.5362597402597402': {11: {'min_than 0.423

In [28]:
fea_dic_2

{10: {'min_than 0.26003119240039696': {21: {'min_than 0.7646968612616297': {2: {'min_than 0.1307490144546649': {43: {'min_than 0.048119041484153566': 'M',
        'max_than 0.048119041484153566': 'R'}},
      'max_than 0.1307490144546649': {3: {'min_than 0.11412268188302425': 'R',
        'max_than 0.11412268188302425': {24: {'min_than 0.3669057377049181': 'R',
          'max_than 0.3669057377049181': 'M'}}}}}},
    'max_than 0.7646968612616297': {41: {'min_than 0.23492063492063497': {8: {'min_than 0.23463645787057602': 'R',
        'max_than 0.23463645787057602': 'M'}},
      'max_than 0.23492063492063497': {30: {'min_than 0.5552588555858311': 'M',
        'max_than 0.5552588555858311': 'R'}}}}}},
  'max_than 0.26003119240039696': {16: {'min_than 0.717231374987048': {34: {'min_than 0.3648869796461082': {17: {'min_than 0.09506493506493506': 'R',
        'max_than 0.09506493506493506': 'M'}},
      'max_than 0.3648869796461082': {17: {'min_than 0.5362597402597402': {11: {'min_than 0.423

In [29]:
fea_dic_3

{10: {'min_than 0.26003119240039696': {21: {'min_than 0.7646968612616297': {2: {'min_than 0.1307490144546649': {43: {'min_than 0.048119041484153566': 'M',
        'max_than 0.048119041484153566': 'R'}},
      'max_than 0.1307490144546649': {3: {'min_than 0.11412268188302425': 'R',
        'max_than 0.11412268188302425': {24: {'min_than 0.3669057377049181': 'R',
          'max_than 0.3669057377049181': 'M'}}}}}},
    'max_than 0.7646968612616297': {41: {'min_than 0.23492063492063497': {8: {'min_than 0.23463645787057602': 'R',
        'max_than 0.23463645787057602': 'M'}},
      'max_than 0.23492063492063497': {30: {'min_than 0.5552588555858311': 'M',
        'max_than 0.5552588555858311': 'R'}}}}}},
  'max_than 0.26003119240039696': {16: {'min_than 0.717231374987048': {34: {'min_than 0.3648869796461082': {17: {'min_than 0.09506493506493506': 'R',
        'max_than 0.09506493506493506': 'M'}},
      'max_than 0.3648869796461082': {17: {'min_than 0.5362597402597402': {11: {'min_than 0.423

In [30]:
fea_dic_4

{10: {'min_than 0.26003119240039696': {21: {'min_than 0.7646968612616297': {2: {'min_than 0.1307490144546649': {43: {'min_than 0.048119041484153566': 'M',
        'max_than 0.048119041484153566': 'R'}},
      'max_than 0.1307490144546649': {3: {'min_than 0.11412268188302425': 'R',
        'max_than 0.11412268188302425': {24: {'min_than 0.3669057377049181': 'R',
          'max_than 0.3669057377049181': 'M'}}}}}},
    'max_than 0.7646968612616297': {41: {'min_than 0.23492063492063497': {8: {'min_than 0.23463645787057602': 'R',
        'max_than 0.23463645787057602': 'M'}},
      'max_than 0.23492063492063497': {30: {'min_than 0.5552588555858311': 'M',
        'max_than 0.5552588555858311': 'R'}}}}}},
  'max_than 0.26003119240039696': {16: {'min_than 0.717231374987048': {34: {'min_than 0.3648869796461082': {17: {'min_than 0.09506493506493506': 'R',
        'max_than 0.09506493506493506': 'M'}},
      'max_than 0.3648869796461082': {17: {'min_than 0.5362597402597402': {11: {'min_than 0.423

# 测试正确率

In [31]:
def test_tree(fea_dic,data):
    fea_dic_keys=list(fea_dic.keys())[0]
    fea_dic=fea_dic[fea_dic_keys]
    
    
    num=data[fea_dic_keys]
    next_node=list(fea_dic.keys())
    num_than=float(re.findall(r"\d+\.?\d*",next_node[0])[0])
    if num<num_than:
        
        char=fea_dic[next_node[0]]
                
        
        if isinstance(char,str):
            return char
        
        if isinstance(char,dict):
            return test_tree(char,data) 
    if num>=num_than:
        
        char=fea_dic[next_node[1]]
       
        if isinstance(char,str):

            return char
           
        
        if isinstance(char,dict):
            return test_tree(char,data)
        
    

In [32]:
def test(fea_dic):
    count=0
    for i in range(50):

        char=str(test_tree(fea_dic,data_test[i,:]))

        if char==label_test[i]:
            count+=1
    print(count/50)

# 四种参数下的正确率

In [33]:
test(fea_dic_0)

0.76


In [34]:
test(fea_dic_1)

0.76


In [35]:
test(fea_dic_2)

0.76


In [36]:
test(fea_dic_3)

0.76


In [37]:
test(fea_dic_4)

0.76


# AdaBoost 

In [38]:
class Get_train_ada(object):
    def __init__(self,data,labels):
        self.data=data
        self.labels=labels
    
    def get_random_number(self):
        list_ram_num=[]
        
        while len(list_ram_num)<70:
            
            num=random.randint(0,120)
            
            list_ram_num.append(num)
            
            list_ram_num=list(set(list_ram_num))
        return np.array(list_ram_num)
        
    def get_Data(self):
        
        random_num=self.get_random_number()
        
        train_ran=list(range(70))
        np.random.shuffle(train_ran)
        data_train=self.data[random_num][train_ran]
        label_train=self.labels[random_num][train_ran]
 
        return data_train,label_train

In [39]:
final_test,final_labels=data_test,label_test

In [40]:
ada_data=data_train[0:121]
ada_labels=label_train[0:121]
ada_datasets=np.concatenate((ada_data,ada_labels.reshape(-1,1)),axis=1)

In [41]:
data_test,label_test=data_train[121:],label_train[121:]

In [42]:
import copy

In [43]:
class forest(object):
    def __init__(self,data,labels,test_data,labels_test,maxiters,final_test,final_labels,number,detle):
        self.data=data
        self.labels=labels
        self.fea_ID=np.array(range(60)).tolist()
        self.w=np.ones(37,)/37
        self.alpha=np.ones(number,)/number
        self.test_data=test_data
        self.labels_test=labels_test
        self.maxiters=maxiters
        self.final_test=final_test
        self.final_labels=final_labels
        self.number=number
        self.detle=detle
        
    
    def get_tree(self,one_data,one_labels):
        one_labels=one_labels.reshape(-1,1)
        data=np.concatenate((one_data,one_labels),axis=1)
        fea_dic=ID3().creatdisiontree(dataset=data,fea_labels=np.array(range(60)).tolist(),detle=self.detle)
        
        
        return fea_dic
    
    def test_tree(self,fea_dic,data):
        fea_dic_keys=list(fea_dic.keys())[0]
        fea_dic=fea_dic[fea_dic_keys]


        num=data[fea_dic_keys]
        next_node=list(fea_dic.keys())
        num_than=float(re.findall(r"\d+\.?\d*",next_node[0])[0])
        if num<num_than:

            char=fea_dic[next_node[0]]


            if isinstance(char,str):
                return char

            if isinstance(char,dict):
                return test_tree(char,data) 
        if num>=num_than:

            char=fea_dic[next_node[1]]

            if isinstance(char,str):

                return char


            if isinstance(char,dict):
                return test_tree(char,data)
    
    def count_truth(self,fea_dic):
        
        count=0
        list_char=[]
        for i in range(37):
    
            char=str(self.test_tree(fea_dic,self.test_data[i,:]))
          
            list_char.append(char)
            if char==self.labels_test[i]:
                count+=1
            
            
        return count/37,list_char
    
    def get_datas(self):
        list_data=[]
        list_labels=[]
        for i in range(self.number):
            data,labels=Get_train_ada(self.data,self.labels).get_Data()
            list_data.append(data)
            list_labels.append(labels)
            
        return list_data,list_labels

    def change_w(self):
        y=copy.copy(self.labels_test)
        y[self.labels_test=='M']=1
        y[self.labels_test=='R']=-1
        y=np.array(list(map(int,y)))
        list_data,list_labels=self.get_datas()
        trees=[]
        for j in range(self.number):
            fea_dic=self.get_tree(list_data[j],list_labels[j])
            trees.append(fea_dic)
        
        for i in range(self.maxiters):
            cout_lis=[]
            list_char=[]
            for j in range(self.number):
                count_tru,char_arr=self.count_truth(trees[j])
        
                cout_lis.append(count_tru)
                list_char.append(char_arr)
            
            index_min=cout_lis.index(max(cout_lis))
            char=list_char[index_min]
            char=np.array(char)
            
            index_where=np.where(char!=self.labels_test)[0]
            
            char[char=='M']=3
            char[char=='R']=1
            char=np.array(list(map(int,char)))-2
            
            loss=float(sum(self.w[index_where]))
            alpha=(1/2)*log((1-loss)/loss)
            self.alpha[index_min]=alpha
            Z=2*np.sqrt(loss*(1-loss))
        
            self.w[index_where]=self.w[index_where]*np.exp(-alpha*y[index_where]*char[index_where])
            
        return trees
    
    def count_truth2(self,fea_dic):

        count=0
        list_char=[]
        for i in range(50):

            char=str(self.test_tree(fea_dic,self.final_test[i,:]))
            list_char.append(char)
            if char==self.final_labels[i]:
                count+=1
        return count/50,np.array(list_char)
    def get_pre(self):
        list_arr=[]
        trees=self.change_w()
        for i in range(self.number):
            count,char_arr=self.count_truth2(trees[i])
            char_arr[char_arr=='M']=3
            char_arr[char_arr=='R']=1
            char_arr=np.array(list(map(int,char_arr)))-2
            list_arr.append(char_arr)
       
        all_=0
        for i in range(self.number):
            all_+=list_arr[i]*self.alpha[i]
        all_=np.array(list(map(str,all_)))
        all_[all_>='0']='M'
        all_[all_<'0']='R'
        
        pre=sum(all_==self.final_labels)/len(self.final_labels)
        return pre
            


# 测试正确率

# 不同棵树下,依次为3，6，9，12

In [44]:
print(forest(ada_data,ada_labels,data_test,label_test,
             maxiters=200,final_test=final_test,final_labels=final_labels,number=3,detle=0.1).get_pre())

0.62


In [45]:
print(forest(ada_data,ada_labels,data_test,label_test,
             maxiters=200,final_test=final_test,final_labels=final_labels,number=6,detle=0.1).get_pre())

0.76


In [46]:
print(forest(ada_data,ada_labels,data_test,label_test,
             maxiters=200,final_test=final_test,final_labels=final_labels,number=9,detle=0.1).get_pre())

0.84


In [47]:
print(forest(ada_data,ada_labels,data_test,label_test,
             maxiters=200,final_test=final_test,final_labels=final_labels,number=12,detle=0.1).get_pre())

0.78


# 不同深度下

In [48]:
print(forest(ada_data,ada_labels,data_test,label_test,
             maxiters=200,final_test=final_test,final_labels=final_labels,number=9,detle=0.1).get_pre())

0.72


In [49]:
print(forest(ada_data,ada_labels,data_test,label_test,
             maxiters=200,final_test=final_test,final_labels=final_labels,number=9,detle=0.11).get_pre())

0.74


In [50]:
print(forest(ada_data,ada_labels,data_test,label_test
             ,maxiters=200,final_test=final_test,final_labels=final_labels,number=9,detle=0.13).get_pre())

0.78


In [51]:
print(forest(ada_data,ada_labels,data_test,label_test,
             maxiters=200,final_test=final_test,final_labels=final_labels,number=9,detle=0.14).get_pre())

0.72
